In [2]:
# Continue from the previous script, assuming 'document_store' is populated.
from scripts.indexing import document_store #this runs our indexing pipeline
# Import additional components for hybrid retrieval
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.joiners import DocumentJoiner
from haystack.components.rankers import TransformersSimilarityRanker

In [3]:
# Import necessary components for the query pipeline
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret
from haystack import Pipeline

In [4]:
# --- 1. Initialize Query Pipeline Components ---

# Text Embedder: To embed the user's query. Must be compatible with the document embedder.
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

# Retriever: Fetches documents from the DocumentStore based on vector similarity.
retriever = InMemoryEmbeddingRetriever(document_store=document_store, top_k=3)

# PromptBuilder: Creates a prompt using the retrieved documents and the query.
# The Jinja2 template iterates through the documents and adds their content to the prompt.
prompt_template_for_pipeline = """
Given the following information, answer the user's question.
If the information is not available in the provided documents, say that you don't have enough information to answer.

Context:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
prompt_builder_inst = PromptBuilder(template=prompt_template_for_pipeline,
                                    required_variables="*")
llm_generator_inst = OpenAIGenerator(api_key=Secret.from_env_var("OPENAI_API_KEY"), model="gpt-4o-mini")



# Sparse Retriever (BM25): For keyword-based search.
# This retriever needs to be "warmed up" by calculating statistics on the documents in the store.
bm25_retriever = InMemoryBM25Retriever(document_store=document_store, top_k=3)

# DocumentJoiner: To merge the results from the two retrievers.
# The default 'concatenate' mode works well here as the ranker will handle final ordering.
document_joiner = DocumentJoiner()

# Ranker: A cross-encoder model to re-rank the combined results for higher precision.
# This model is highly effective at identifying the most relevant documents from a candidate set.
ranker = TransformersSimilarityRanker(model="BAAI/bge-reranker-base", top_k=3)



TransformersSimilarityRanker is considered legacy and will no longer receive updates. It may be deprecated in a future release, with removal following after a deprecation period. Consider using SentenceTransformersSimilarityRanker instead, which provides the same functionality along with additional features.


In [5]:
# --- 2. Build the Hybrid RAG Pipeline ---

hybrid_rag_pipeline = Pipeline()

# Add all necessary components
hybrid_rag_pipeline.add_component("text_embedder", text_embedder)
hybrid_rag_pipeline.add_component("embedding_retriever", retriever) # Dense retriever
hybrid_rag_pipeline.add_component("bm25_retriever", bm25_retriever) # Sparse retriever
hybrid_rag_pipeline.add_component("document_joiner", document_joiner)
hybrid_rag_pipeline.add_component("ranker", ranker)
hybrid_rag_pipeline.add_component("prompt_builder", prompt_builder_inst)
hybrid_rag_pipeline.add_component("llm", llm_generator_inst)

In [8]:
# --- 3. Connect the Components in a Graph ---

# The query is embedded for the dense retriever
hybrid_rag_pipeline.connect("text_embedder.embedding", "embedding_retriever.query_embedding")

# The raw query text is sent to the BM25 retriever and the ranker
# Note: The query input for these components is the raw text string.

# The outputs of both retrievers are fed into the document joiner
hybrid_rag_pipeline.connect("embedding_retriever.documents", "document_joiner.documents")
hybrid_rag_pipeline.connect("bm25_retriever.documents", "document_joiner.documents")

# The joined documents are sent to the ranker
hybrid_rag_pipeline.connect("document_joiner.documents", "ranker.documents")

# The ranked documents are sent to the prompt builder
hybrid_rag_pipeline.connect("ranker.documents", "prompt_builder.documents")

# The final prompt is sent to the LLM
hybrid_rag_pipeline.connect("prompt_builder.prompt", "llm.prompt")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - embedding_retriever: InMemoryEmbeddingRetriever
  - bm25_retriever: InMemoryBM25Retriever
  - document_joiner: DocumentJoiner
  - ranker: TransformersSimilarityRanker
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> embedding_retriever.query_embedding (list[float])
  - embedding_retriever.documents -> document_joiner.documents (list[Document])
  - bm25_retriever.documents -> document_joiner.documents (list[Document])
  - document_joiner.documents -> ranker.documents (list[Document])
  - ranker.documents -> prompt_builder.documents (list[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [9]:
# --- 4. Visualize the Pipeline (Optional) ---
try:
    hybrid_rag_pipeline.draw(path="./images/hybrid_rag_pipeline.png")
    print("Hybrid pipeline visualization saved to 'hybrid_rag_pipeline.png'")
except Exception as e:
    print(f"Could not draw hybrid pipeline: {e}")

Hybrid pipeline visualization saved to 'hybrid_rag_pipeline.png'


![](./images/hybrid_rag_pipeline.png)

In [ ]:
# --- 5. Run the Pipeline ---

# A query that benefits from both semantic and keyword matching
hybrid_question = "What is the Haystack 2.0 framework?"

# The run dictionary must now provide inputs for all components at the start of the graph.
# The query text needs to be passed to the text_embedder, bm25_retriever, ranker, and prompt_builder.
hybrid_result = hybrid_rag_pipeline.run({
    "text_embedder": {"text": hybrid_question},
    "bm25_retriever": {"query": hybrid_question},
    "ranker": {"query": hybrid_question},
    "prompt_builder": {"question": hybrid_question}
})

print(f"\nQuestion: {hybrid_question}")
print(f"Answer: {hybrid_result['llm']['replies'].content}")

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.69s/it]
